In [13]:
import os

api_key = os.getenv("DEEPSEEK_API_KEY")

In [14]:
import re
import os
from pathlib import Path
from typing import List, Dict

text_lines = []
text = ""
with open("mfd.md", 'r', encoding='utf-8') as f:
    text = f.read()

lines = text.splitlines()
current_path = []
law_chunks = []

article_pattern = re.compile(r'^第[一二三四五六七八九十百千万零〇\d]+条')
current_article = ""

for line in lines:
    line = line.strip()
    if not line:
        continue
    line = re.sub(r'\*\*', '', line).strip()
    if article_pattern.match(line):
        if current_article:
            text_lines.append(current_article)
        current_article = line
    else:
        current_article += "\n" + line

if current_article:
    text_lines.append(current_article)



In [15]:
len(text_lines)

388

In [16]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

In [17]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

In [18]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)

768


In [19]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "mfd_rag_collection"

In [20]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

In [21]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # 内积距离
    consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
)

In [22]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 388/388 [00:00<00:00, 531479.41it/s]


{'insert_count': 388, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 

In [23]:
question = "借用他人的物品，损坏了该怎么赔？"

In [24]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),
    limit=3,
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [25]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "\u7b2c\u4e8c\u767e\u4e8c\u5341\u4e09\u6761 \u52a8\u4ea7\u7269\u6743\u7684\u8bbe\u7acb\u548c\u8f6c\u8ba9\uff0c\u81ea\u4ea4\u4ed8\u65f6\u53d1\u751f\u6548\u529b\uff0c\u4f46\u662f\u6cd5\u5f8b\u53e6\u6709\u89c4\u5b9a\u7684\u9664\u5916\u3002",
        0.8118640184402466
    ],
    [
        "\u7b2c\u4e8c\u767e\u4e00\u5341\u56db\u6761 \u4e0d\u52a8\u4ea7\u7269\u6743\u7684\u8bbe\u7acb\u3001\u53d8\u66f4\u3001\u8f6c\u8ba9\u548c\u6d88\u706d\uff0c\u4f9d\u7167\u6cd5\u5f8b\u89c4\u5b9a\u5e94\u5f53\u767b\u8bb0\u7684\uff0c\u81ea\u8bb0\u8f7d\u4e8e\u4e0d\u52a8\u4ea7\u767b\u8bb0\u7c3f\u65f6\u53d1\u751f\u6548\u529b\u3002",
        0.8118640184402466
    ],
    [
        "\u7b2c\u4e8c\u767e\u96f6\u516b\u6761 \u4e0d\u52a8\u4ea7\u6743\u5229\u7684\u8bbe\u7acb\u3001\u53d8\u66f4\u3001\u8f6c\u8ba9\u548c\u6d88\u706d\uff0c\u5e94\u5f53\u4f9d\u7167\u6cd5\u5f8b\u89c4\u5b9a\u767b\u8bb0\u3002\u52a8\u4ea7\u7269\u6743\u7684\u8bbe\u7acb\u548c\u8f6c\u8ba9\uff0c\u5e94\u5f53\u4f9d\u7167\u6cd5\u5f8b\u89c4\u5b9a

In [26]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [27]:
context

'第二百二十三条 动产物权的设立和转让，自交付时发生效力，但是法律另有规定的除外。\n第二百一十四条 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。\n第二百零八条 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。'

In [28]:
question

'借用他人的物品，损坏了该怎么赔？'

In [29]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [30]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。\n<context>\n第二百二十三条 动产物权的设立和转让，自交付时发生效力，但是法律另有规定的除外。\n第二百一十四条 不动产物权的设立、变更、转让和消灭，依照法律规定应当登记的，自记载于不动产登记簿时发生效力。\n第二百零八条 不动产权利的设立、变更、转让和消灭，应当依照法律规定登记。动产物权的设立和转让，应当依照法律规定交付。\n</context>\n<question>\n借用他人的物品，损坏了该怎么赔？\n</question>\n<translated>\n</translated>\n'

In [31]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的法律条文上下文，关于借用物品损坏赔偿问题需注意：

1. 动产（如借用物品）的物权变动以交付为生效要件（第223条）
2. 虽然条文未直接规定损害赔偿，但依据第208条"动产物权的设立和转让应当依法交付"可推知：
   - 借用关系成立时即产生保管义务
   - 损坏他人动产应承担侵权责任或违约责任
   - 具体赔偿标准需结合《民法典》侵权责任编相关规定

建议处理方式：
- 优先协商修复或等价赔偿
- 协商不成可按损害发生时市场价赔偿
- 若属故意损坏可能涉及惩罚性赔偿

<translated>
根据提供的法律条文上下文，关于借用物品损坏赔偿问题需注意：

1. 动产（如出借物品）的物权变动以交付为生效要件（第223条）  
2. 虽然条文未直接规定损害赔偿，但依据第208条"动产物权的设立和转让应当依法交付"可推知：
   - 借贷关系成立时即产生保管义务
   - 损坏他人动产应承担侵权责任或违约责任  
   - 具体赔偿标准需结合《民法典》侵权责任编相关规定

建议处理方式：
- 优先协商修复或等价赔偿  
- 协商不成可按损害发生时市场价赔偿
- 若属故意损坏可能涉及惩罚性赔偿
</translated>


In [32]:
question = "农村里的宅基地属于谁?"

In [33]:
question

'农村里的宅基地属于谁?'

In [34]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),
    limit=3,
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

In [35]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

[
    [
        "\u7b2c\u4e09\u767e\u4e00\u5341\u56db\u6761 \u6309\u4efd\u5171\u6709\u4eba\u5bf9\u5171\u6709\u7684\u4e0d\u52a8\u4ea7\u6216\u8005\u52a8\u4ea7\u6309\u7167\u5176\u4efd\u989d\u4eab\u6709\u6240\u6709\u6743\u3002",
        0.7870465517044067
    ],
    [
        "\u7b2c\u4e8c\u767e\u4e03\u5341\u4e03\u6761 \u4e1a\u4e3b\u4e0d\u5f97\u4ee5\u653e\u5f03\u6743\u5229\u4e3a\u7531\u4e0d\u5c65\u884c\u4e49\u52a1\u3002",
        0.7870465517044067
    ],
    [
        "\u7b2c\u4e8c\u767e\u96f6\u4e94\u6761 \u672c\u7f16\u8c03\u6574\u56e0\u7269\u7684\u5f52\u5c5e\u548c\u5229\u7528\u4ea7\u751f\u7684\u6c11\u4e8b\u5173\u7cfb\u3002",
        0.7870465517044067
    ]
]


In [36]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

In [37]:
context

'第三百一十四条 按份共有人对共有的不动产或者动产按照其份额享有所有权。\n第二百七十七条 业主不得以放弃权利为由不履行义务。\n第二百零五条 本编调整因物的归属和利用产生的民事关系。'

In [39]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

In [40]:
USER_PROMPT

'\n请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。\n<context>\n第三百一十四条 按份共有人对共有的不动产或者动产按照其份额享有所有权。\n第二百七十七条 业主不得以放弃权利为由不履行义务。\n第二百零五条 本编调整因物的归属和利用产生的民事关系。\n</context>\n<question>\n农村里的宅基地属于谁?\n</question>\n<translated>\n</translated>\n'

In [41]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文，虽然未直接提及"宅基地"这一具体概念，但可以从以下法律条款推导相关原则：
1. 依据《民法典》第二百零五条，物的归属问题属于民事关系调整范畴
2. 第三百一十四条规定的按份共有制度可能适用于宅基地共有情况
3. 第二百七十七条特别强调权利与义务的对等性，暗示宅基地使用权人不得随意放弃相关义务

（注：由于上下文未提供宅基地权属的直接规定，建议补充《土地管理法》等相关法律条文以获得更准确答案）

<translated>
根据提供的上下文，虽然未直接提及"宅基地"这一具体概念，但可以从以下法律条款推导相关原则：
1. 依据《民法典》第205条，物的归属问题属于民事关系调整范畴  
2. 第314条规定的按份共有制度可能适用于宅基地共有情况  
3. 第277条特别强调权利与义务的对等性，暗示宅基地使用权人不得随意放弃相关义务  

（注：由于上下文未提供宅基地权属的直接规定，建议补充《土地管理法》等相关法律条文以获得更准确答案）
</translated>
